In [81]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
pd.options.display.max_columns = 999
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import seaborn as sns

# Load Data 

In [44]:
df = pd.read_csv("AmesHousing.tsv",delimiter="\t")
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [45]:
def transform_features(df):
    return df

def select_features(df):
    return df[["Gr Liv Area","SalePrice"]]

def train_test(df):
    model = LinearRegression()
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=["integer","float"])
    numeric_test = test.select_dtypes(include=["integer","float"])
    numeric_train = numeric_train.drop("SalePrice",axis=1)
    numeric_test = numeric_test.drop("SalePrice",axis=1)
    
    model.fit(numeric_train,train["SalePrice"])
    predictions = model.predict(numeric_test)
    error = mean_squared_error(test["SalePrice"],predictions)
    rmse = np.sqrt(error)
    
    return rmse
    
    
    

## Feature Engineering 

In [46]:
missing = df.isnull().sum()/len(df) * 100
miss_less_5 = missing[(missing < 5) & (missing > 0)].index
miss_more_25 = missing[missing > 25].index


### Drop columns with more than 25% missing value 

In [47]:
df = df.drop(miss_more_25,axis = 1)
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,3,2010,WD,Normal,189900


### Handling missing text columns 

In [48]:
missing_text = df.select_dtypes(include=["object"]).isnull().sum()/len(df) * 100
missing_text = missing_text[missing_text > 0].index
missing_text

Index(['Mas Vnr Type', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Electrical', 'Garage Type',
       'Garage Finish', 'Garage Qual', 'Garage Cond'],
      dtype='object')

#### drop missing text column 

In [49]:
df = df.drop(missing_text,axis = 1)
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,3,2010,WD,Normal,189900


### Handling Numerical Missing Value 

In [50]:
numeric_data = df.select_dtypes(include=["integer","float"])
missing_numeric = df.select_dtypes(include=["integer","float"]).isnull().sum()/len(df) * 100
missing_numeric = missing_numeric[missing_numeric > 0].index

for col in missing_numeric:
    df[col] = df[col].fillna(df[col].mode()[0])
df.isnull().sum().value_counts()

0    66
dtype: int64

### Create New Features 

In [52]:
years_sold = df["Yr Sold"] - df["Year Built"]

# Check for some negative value
years_sold [years_sold < 0]

2180   -1
dtype: int64

In [53]:
years_since_remod = df["Yr Sold"] - df["Year Remod/Add"]

years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [54]:
df["House_age"] = years_sold
df["House_age_remod"] = years_since_remod

df = df.drop([2180,1702,2180,2181],axis=0)

df = df.drop(["Year Built","Year Remod/Add"],axis=1)

### Drop columns that not useful for ml and potentially data leakage 

In [55]:
df = df.drop(["PID","Order"],axis = 1)
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

### Updating Function 

In [58]:
def transform_features(df):
    missing = df.isnull().sum()/len(df) * 100
    miss_less_5 = missing[(missing < 5) & (missing > 0)].index
    miss_more_25 = missing[missing > 25].index
    df = df.drop(miss_more_25,axis=1)
    
    missing_text = df.select_dtypes(include=["object"]).isnull().sum()/len(df) * 100
    missing_text = missing_text[missing_text > 0].index
    df = df.drop(missing_text,axis=1)
    
    numeric_data = df.select_dtypes(include=["integer","float"])
    missing_numeric = df.select_dtypes(include=["integer","float"]).isnull().sum()/len(df) * 100
    missing_numeric = missing_numeric[missing_numeric > 0].index

    for col in missing_numeric:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    years_sold = df["Yr Sold"] - df["Year Built"]
    years_since_remod = df["Yr Sold"] - df["Year Remod/Add"]
    
    df["House_age"] = years_sold
    df["House_age_remod"] = years_since_remod
    df = df.drop([2180,1702,2180,2181],axis=0)
    df = df.drop(["Year Built","Year Remod/Add"],axis=1)
    
    df = df.drop(["PID","Order"],axis = 1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

def select_features(df):
    return df[["Gr Liv Area","SalePrice"]]

def train_test(df):
    model = LinearRegression()
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=["integer","float"])
    numeric_test = test.select_dtypes(include=["integer","float"])
    numeric_train = numeric_train.drop("SalePrice",axis=1)
    numeric_test = numeric_test.drop("SalePrice",axis=1)
    
    model.fit(numeric_train,train["SalePrice"])
    predictions = model.predict(numeric_test)
    error = mean_squared_error(test["SalePrice"],predictions)
    rmse = np.sqrt(error)
    
    return rmse
    

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_test(filtered_df)

rmse
    

55275.36731241307

# Feature Selection 

In [66]:
numeric_data = transform_df.select_dtypes(include=["integer","float"])
corr_saleprice = numeric_data.corr()["SalePrice"].abs().sort_values(ascending=False)
corr_saleprice

SalePrice          1.000000
Overall Qual       0.801206
Gr Liv Area        0.717596
Garage Cars        0.648361
Total Bsmt SF      0.644012
Garage Area        0.641425
1st Flr SF         0.635185
House_age          0.558979
Full Bath          0.546118
House_age_remod    0.534985
Mas Vnr Area       0.506983
TotRms AbvGrd      0.498574
Fireplaces         0.474831
Garage Yr Blt      0.442216
BsmtFin SF 1       0.439284
Lot Frontage       0.333681
Wood Deck SF       0.328183
Open Porch SF      0.316262
Half Bath          0.284871
Bsmt Full Bath     0.276258
2nd Flr SF         0.269601
Lot Area           0.267520
Bsmt Unf SF        0.182751
Bedroom AbvGr      0.143916
Enclosed Porch     0.128685
Kitchen AbvGr      0.119760
Screen Porch       0.112280
Overall Cond       0.101540
MS SubClass        0.085128
Pool Area          0.068438
Low Qual Fin SF    0.037629
Bsmt Half Bath     0.035875
3Ssn Porch         0.032268
Misc Val           0.019273
BsmtFin SF 2       0.006127
Name: SalePrice, dty

In [67]:
corr_saleprice[corr_saleprice > 0.4]

SalePrice          1.000000
Overall Qual       0.801206
Gr Liv Area        0.717596
Garage Cars        0.648361
Total Bsmt SF      0.644012
Garage Area        0.641425
1st Flr SF         0.635185
House_age          0.558979
Full Bath          0.546118
House_age_remod    0.534985
Mas Vnr Area       0.506983
TotRms AbvGrd      0.498574
Fireplaces         0.474831
Garage Yr Blt      0.442216
BsmtFin SF 1       0.439284
Name: SalePrice, dtype: float64

In [79]:
transform_df = transform_df.drop(corr_saleprice[corr_saleprice <0.4].index,axis=1)

In [75]:
text_data = transform_df.select_dtypes(include=["object"])
num_unique = text_data.nunique().sort_values(ascending=True)
unique_less_10 = num_unique[num_unique < 10].index
unique_more_10 = num_unique[num_unique > 10].index
transform_df = transform_df.drop(unique_more_10,axis=1)


In [76]:
for col in unique_less_10 :
    print("{0}: {1}".format(col,transform_df[col].value_counts(normalize=True)*100))

Street: Pave    99.590024
Grvl     0.409976
Name: Street, dtype: float64
Central Air: Y    93.303724
N     6.696276
Name: Central Air, dtype: float64
Paved Drive: Y    90.502221
N     7.379570
P     2.118210
Name: Paved Drive, dtype: float64
Utilities: AllPub    99.897506
NoSewr     0.068329
NoSeWa     0.034165
Name: Utilities, dtype: float64
Land Slope: Gtl    95.182781
Mod     4.270584
Sev     0.546635
Name: Land Slope, dtype: float64
Lot Shape: Reg    63.512128
IR1    33.344722
IR2     2.596515
IR3     0.546635
Name: Lot Shape, dtype: float64
Land Contour: Lvl    89.921421
HLS     4.099761
Bnk     3.928937
Low     2.049880
Name: Land Contour, dtype: float64
Exter Qual: TA    61.462248
Gd    33.754698
Ex     3.587291
Fa     1.195764
Name: Exter Qual, dtype: float64
Lot Config: Inside     73.044072
Corner     17.423984
CulDSac     6.149641
FR2         2.903997
FR3         0.478305
Name: Lot Config, dtype: float64
Kitchen Qual: TA    51.042023
Gd    39.596857
Ex     6.935429
Fa     2.3

In [77]:
# low variance Data
low_var = ["Street","Utilities","Land Slope","Heating","Roof Matl","Condition 2"]
transform_df = transform_df.drop(low_var,axis=1)

In [78]:
text_cols = transform_df.select_dtypes(include=["object"]).columns

for col in text_cols:
    transform_df[col] = transform_df[col].astype("category")

category_data = transform_df.select_dtypes(include=["category"])
transform_df = pd.concat([transform_df,pd.get_dummies(category_data)],axis=1).drop(text_cols,axis=1)


## Update Function 

In [ ]:
def transform_features(df):
    missing = df.isnull().sum()/len(df) * 100
    miss_less_5 = missing[(missing < 5) & (missing > 0)].index
    miss_more_25 = missing[missing > 25].index
    df = df.drop(miss_more_25,axis=1)
    
    missing_text = df.select_dtypes(include=["object"]).isnull().sum()/len(df) * 100
    missing_text = missing_text[missing_text > 0].index
    df = df.drop(missing_text,axis=1)
    
    numeric_data = df.select_dtypes(include=["integer","float"])
    missing_numeric = df.select_dtypes(include=["integer","float"]).isnull().sum()/len(df) * 100
    missing_numeric = missing_numeric[missing_numeric > 0].index

    for col in missing_numeric:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    years_sold = df["Yr Sold"] - df["Year Built"]
    years_since_remod = df["Yr Sold"] - df["Year Remod/Add"]
    
    df["House_age"] = years_sold
    df["House_age_remod"] = years_since_remod
    df = df.drop([2180,1702,2180,2181],axis=0)
    df = df.drop(["Year Built","Year Remod/Add"],axis=1)
    
    df = df.drop(["PID","Order"],axis = 1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

def select_features(df):
    numeric_data = df.select_dtypes(include=["integer","float"])
    corr_saleprice = numeric_data.corr()["SalePrice"].abs().sort_values(ascending=False)
    df = df.drop(corr_saleprice[corr_saleprice <0.4].index,axis=1)
    text_data = df.select_dtypes(include=["object"])
    num_unique = text_data.nunique().sort_values(ascending=True)
    unique_less_10 = num_unique[num_unique < 10].index
    unique_more_10 = num_unique[num_unique > 10].index
    df = transform_df.drop(unique_more_10,axis=1)
    low_var = ["Street","Utilities","Land Slope","Heating","Roof Matl","Condition 2"]
    df = df.drop(low_var,axis=1)
    
    text_cols = df.select_dtypes(include=["object"]).columns

    for col in text_cols:
        df[col] = df[col].astype("category")

    category_data = df.select_dtypes(include=["category"])
    df = pd.concat([df,pd.get_dummies(category_data)],axis=1).drop(text_cols,axis=1)

    return df

def train_test(df,k=0):
    model = LinearRegression()
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    
    
    if k==0:
        
        train = df[:1460]
        test = df[1460:]

        model.fit(train[features], train["SalePrice"])
        predictions = model.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k==1:
        shuffled_df = df.sample(frac=1,)
        fold_one= df[:1460]
        fold_two = df[1460:]
        
        model.fit(fold_one[features],fold_one["SalePrice"])
        predictions = model.predict(fold_two[features])
        error_one = mean_squared_error(fold_two["SalePrice"],predictions)
        rmse_one = np.sqrt(error_one)
        
        model.fit(fold_two[features],fold_two["SalePrice"])
        predictions = model.predict(fold_one[features])
        error_two = mean_squared_error(fold_one["SalePrice"],predictions)
        rmse_two = np.sqrt(error_two)
        
        rmses = [rmse_one,rmse_two]
        
        return rmses.mean()
    else:
        kf = KFold(n_splits = k)
        rmses = []
        for train_index,test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            model.fit(train[features],train["SalePrice"])
            result = model.predict(test[features])
            error = mean_squared_error(test["SalePrice"],result)
            rmse = np.sqrt(error)
            rmses.append(rmse)
        return rmses.mean
    

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_test(filtered_df)

rmse
    